# Synthetic Data Generation (Encoding/Decoding) using Auto Encoders
#### Contractive autoencoders
Author: Gabriel Rodrigues Palma

Purpose: Encode and decode 5G data. Also, generate new dasets based on the autoencoders methods.

Sourcers: Contractive autoencoders code based on the paper: 

$\textbf{Contractive Auto-Encoders: Explicit Invariance During Feature Extraction}$ 

$\textit{Salah Rifai, Pascal Vincent, Xavier Muller, Xavier Glorot, and Yoshua Bengio. Contractive auto-encoders: Explicit invariance during feature extraction. In Proceedings of the 28th International Conference on International Conference on Machine Learning, ICML’11, pages 833–840, USA, 2011. Omnipress.}$

# Packages

In [1]:
# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keract import get_activations, display_activations

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Main functions

In [13]:
def transformCategoricalToBinary(data):
    '''Function created to transform categorical data to binary arrays'''
    reshapedData = np.array(data).reshape(len(data), -1)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(reshapedData)    
    return(onehot_encoded)

def transformBinaryToCategorical(data, query):
    '''Function created to transform binary arrays data to categorical'''
    reshapedData = np.array(data).reshape(len(data), -1)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(reshapedData)   
    query = query

    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(list(np.unique(reshapedData)))
    inverted = label_encoder.inverse_transform(list(np.argmax(query, axis=1))) 
    return(inverted)

def transformScaledToNonscaled(data, query):
    '''Function to obtain original data from scaled one'''
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(data) 
    if query.ndim ==1:
        query = query.reshape(1, len(query))
    else:
        query = query.reshape(len(query), -1)
    result = scaler.inverse_transform(query)

    return(result)

def getOriginalData(original_data, categorical_data, decoded_data):
    '''Function to obtain original data from decoded data'''
    decoded_numerical_data = decoded_data[:, 0:8]
    decoded_categorical_data = np.round(decoded_data[:, 8:42])
    numerical_data = transformScaledToNonscaled(data = original_data, 
                                                query = decoded_numerical_data)
    categorical_data = transformBinaryToCategorical(data = categorical_data, 
                                                    query = decoded_categorical_data)    
    result = np.hstack((numerical_data, categorical_data.reshape(len(categorical_data), 1)))

    return(result)
def getOriginalNumericalData(original_data, categorical_data, decoded_data):
    '''Function to obtain original data from decoded data'''
    decoded_numerical_data = decoded_data   
    numerical_data = transformScaledToNonscaled(data = original_data, 
                                                query = decoded_numerical_data)
    result = numerical_data

    return(result)
def getOriginalCategoricalData(original_data, categorical_data, decoded_data):
    '''Function to obtain original data from decoded data'''
    decoded_categorical_data = np.round(decoded_data)
    categorical_data = transformBinaryToCategorical(data = categorical_data, 
                                                    query = decoded_categorical_data)  
    result = numerical_data

    return(result)

# Loading data

In [3]:
original_data = pd.read_csv('../../../Data/5G-production-dataset/Download/Static/B_2019.12.16_13.40.04.csv')
encoded_text_data = transformCategoricalToBinary(original_data['PINGMAX'])
original_data.describe()

,Longitude,Latitude,Speed,CellID,RSRP,RSRQ,SNR,CQI,DL_bitrate,UL_bitrate,RAWCELLID
count,2.143000e+03,2.143000e+03,2143.0,2143.0,2143.000000,2143.000000,2143.000000,2143.00000,2143.000000,2143.000000,2143.0
mean,-8.394628e+00,5.188614e+01,0.0,11.0,-95.999067,-13.856276,3.717219,10.65329,50382.109193,141.879608,10805003.0
std,1.221694e-07,9.162706e-08,0.0,0.0,1.274617,2.239170,3.260739,1.91965,65620.519457,122.211380,0.0
min,-8.394628e+00,5.188614e+01,0.0,11.0,-104.000000,-19.000000,-7.000000,6.00000,0.000000,0.000000,10805003.0
25%,-8.394628e+00,5.188614e+01,0.0,11.0,-96.000000,-16.000000,2.000000,10.00000,3229.500000,44.000000,10805003.0
50%,-8.394628e+00,5.188614e+01,0.0,11.0,-96.000000,-14.000000,4.000000,11.00000,23853.000000,128.000000,10805003.0
75%,-8.394628e+00,5.188614e+01,0.0,11.0,-95.000000,-12.000000,6.000000,11.00000,61175.000000,215.000000,10805003.0
max,-8.394624e+00,5.188614e+01,0.0,11.0,-93.000000,-9.000000,15.000000,15.00000,302694.000000,1215.000000,10805003.0


In [4]:
encoded_text_data_test = transformCategoricalToBinary(np.unique(original_data['PINGMAX']))

In [5]:
selected_data = original_data[['CellID','RSRP','RSRQ','SNR','CQI','DL_bitrate','UL_bitrate','RAWCELLID']]
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(selected_data)
data_scaled = scaler.transform(selected_data)

In [6]:
data = np.hstack((data_scaled, encoded_text_data))
X_train, X_test, y_train, y_test = train_test_split(data, data, test_size=0.3, random_state=42)

# Auto encoder architecture

In [7]:
lam = 0.01
# Start with a sequential model
autoencoder = Sequential()

# Add a dense layer with input the original image pixels and neurons the encoded representation
autoencoder.add(Dense(30, input_shape=(8, )))
autoencoder.add(Dense(20, input_shape=(42, )))
# Add an output layer with as many neurons as the orginal image pixels
autoencoder.add(Dense(8, activation = "sigmoid"))

# Compile your model with adadelta
autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics = 'accuracy')

# Summarize your model structure
autoencoder.summary()

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                270       
                                                                 
 dense_1 (Dense)             (None, 20)                620       
                                                                 
 dense_2 (Dense)             (None, 8)                 168       
                                                                 
Total params: 1,058
Trainable params: 1,058
Non-trainable params: 0
_________________________________________________________________


2022-07-12 13:45:49.625582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-12 13:45:49.625810: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
autoencoder.fit(X_train[:, 0:8], X_train[:, 0:8], epochs = 200)

Epoch 1/200


2022-07-12 13:45:55.084105: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-12 13:45:55.258143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 1s 5ms/step - loss: 0.7569 - accuracy: 0.0100
Epoch 2/200
47/47 [==============================] - 0s 4ms/step - loss: 0.7563 - accuracy: 0.0100
Epoch 3/200
47/47 [==============================] - 0s 4ms/step - loss: 0.7556 - accuracy: 0.0107
Epoch 4/200
47/47 [==============================] - 0s 4ms/step - loss: 0.7549 - accuracy: 0.0107
Epoch 5/200
47/47 [==============================] - 0s 4ms/step - loss: 0.7542 - accuracy: 0.0107
Epoch 6/200
47/47 [==============================] - 0s 4ms/step - loss: 0.7535 - accuracy: 0.0107
Epoch 7/200
47/47 [==============================] - 0s 4ms/step - loss: 0.7528 - accuracy: 0.0107
Epoch 8/200
47/47 [==============================] - 0s 5ms/step - loss: 0.7520 - accuracy: 0.0107
Epoch 9/200
47/47 [==============================] - 0s 5ms/step - loss: 0.7512 - accuracy: 0.0107
Epoch 10/200
47/47 [==============================] - 0s 5ms/step - loss: 0.7504 - accuracy: 0.0107
Epoch 11/200
47/47 [=

## Obtaining numerical encoded and decoded data

In [9]:
# Build your encoder by using the first layer of your autoencoder
encoder = Sequential()
encoder.add(autoencoder.layers[0])

# Encode the noisy images and show the encodings for your favorite number [0-9]
encoded_data = encoder.predict(X_train[:,0:8])

#show_encodings(encodings, number = 1)
# Predict on the noisy images with your autoencoder
decoded_data = autoencoder.predict(X_train[:,0:8])

47/47 [==============================] - 0s 1ms/step


2022-07-12 14:08:41.805168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-12 14:08:41.896199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


# Showing autoencoder results

## Original data

In [10]:
original_csv = getOriginalData(original_data = selected_data, 
                              categorical_data = original_data['PINGMAX'], 
                              decoded_data = X_train)
original_csv = pd.DataFrame(original_csv, 
             columns = ['CellID','RSRP','RSRQ','SNR','CQI','DL_bitrate',
                        'UL_bitrate','RAWCELLID', 'PINGMAX'])
original_csv

,CellID,RSRP,RSRQ,SNR,CQI,DL_bitrate,UL_bitrate,RAWCELLID,PINGMAX
0,11.0,-95.99999999999999,-17.0,2.9999999999999996,8.0,61756.0,256.0,10805003.0,-
1,11.0,-97.0,-16.0,4.999999999999999,8.0,25025.0,145.0,10805003.0,-
2,11.0,-95.99999999999999,-12.000000000000002,7.0,10.000000000000002,6614.999999999999,27.999999999999996,10805003.0,79.0
3,11.0,-95.0,-15.0,4.0,11.0,36005.0,149.0,10805003.0,-
4,11.0,-95.0,-16.0,2.9999999999999996,10.000000000000002,36367.0,136.0,10805003.0,-
...,...,...,...,...,...,...,...,...,...
1495,11.0,-95.99999999999999,-15.0,1.9999999999999993,11.0,195371.0,281.0,10805003.0,-
1496,11.0,-95.99999999999999,-16.0,8.000000000000002,8.0,0.0,0.0,10805003.0,76.0
1497,11.0,-95.99999999999999,-12.000000000000002,4.999999999999999,10.000000000000002,31222.000000000004,138.0,10805003.0,-
1498,11.0,-95.0,-12.000000000000002,1.0000000000000002,11.0,10180.0,125.0,10805003.0,-


## Encoded data

In [11]:
pd.DataFrame(encoded_data, 
             columns = ['Encode%s'%(number) for number in range(encoded_data.shape[1])])

,Encode0,Encode1,Encode2,Encode3,Encode4,Encode5,Encode6,Encode7,Encode8,Encode9,...,Encode20,Encode21,Encode22,Encode23,Encode24,Encode25,Encode26,Encode27,Encode28,Encode29
0,0.162363,0.039636,0.507975,-0.178750,-0.032879,-0.133882,0.046354,0.309284,-0.293497,0.042558,...,0.036838,-0.021133,0.148603,-0.185396,0.098135,-0.157162,0.292086,-0.381707,0.209999,0.077881
1,0.072123,0.018391,0.501548,-0.182824,-0.069275,-0.162785,0.140587,0.352908,-0.303448,0.091091,...,0.129996,-0.042667,0.089736,-0.185908,0.113065,-0.195806,0.348472,-0.353341,0.247319,0.036808
2,0.023469,0.088363,0.585284,-0.151924,-0.036347,-0.308863,0.215320,0.553400,-0.429604,0.201402,...,0.317060,-0.099394,0.211136,-0.192320,0.183779,-0.271462,0.493542,-0.301709,0.341885,-0.086022
3,0.146254,0.081768,0.576707,-0.145632,-0.064093,-0.272514,0.113942,0.460294,-0.415193,0.109222,...,0.217644,-0.086467,0.241189,-0.147065,0.201180,-0.281149,0.407387,-0.372273,0.247408,-0.050633
4,0.156914,0.060496,0.552794,-0.160279,-0.085974,-0.213909,0.091266,0.414503,-0.383474,0.074049,...,0.152941,-0.096347,0.215155,-0.159429,0.182461,-0.257497,0.383801,-0.380959,0.200594,0.008105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.219013,0.086365,0.618287,0.028912,0.182945,-0.291862,-0.088329,0.398250,-0.445843,0.157510,...,0.147132,0.081625,0.442986,-0.197034,0.023039,-0.080728,0.168707,-0.332997,0.341209,-0.035457
1496,0.028706,-0.043862,0.589992,-0.225591,-0.180676,-0.181540,0.224955,0.446141,-0.386327,0.113284,...,0.192943,-0.106596,0.047414,-0.243986,0.157219,-0.277454,0.451187,-0.424380,0.252036,0.077204
1497,0.075100,0.133910,0.550731,-0.131862,0.037422,-0.296104,0.143571,0.506448,-0.391409,0.179904,...,0.259107,-0.063211,0.261013,-0.168192,0.159387,-0.218000,0.434873,-0.273457,0.334484,-0.094764
1498,0.137754,0.201103,0.500106,-0.123178,0.025296,-0.292007,0.097081,0.502705,-0.382467,0.135715,...,0.241373,-0.131184,0.339818,-0.102709,0.221321,-0.251729,0.456825,-0.234408,0.242009,-0.126242


## Decoded data

In [14]:
decoded_csv = getOriginalNumericalData(original_data = selected_data, 
                              categorical_data = original_data['PINGMAX'], 
                              decoded_data = decoded_data)
decoded_csv = pd.DataFrame(decoded_csv, 
             columns = ['CellID','RSRP','RSRQ','SNR','CQI','DL_bitrate',
                        'UL_bitrate','RAWCELLID'])
decoded_csv

,CellID,RSRP,RSRQ,SNR,CQI,DL_bitrate,UL_bitrate,RAWCELLID
0,11.335633,-97.458664,-14.176364,5.316362,11.139453,126593.414062,400.158295,10805003.0
1,11.329069,-97.280022,-14.391945,4.681108,11.054911,126223.414062,405.284027,10805003.0
2,11.283361,-96.726936,-14.681438,4.509181,11.497167,117638.007812,373.234955,10805003.0
3,11.312831,-96.993027,-14.423570,5.218916,11.420067,116078.242188,384.350891,10805003.0
4,11.328897,-97.115829,-14.330244,5.336322,11.287884,119475.390625,397.227722,10805003.0
...,...,...,...,...,...,...,...,...
1495,11.285090,-97.606369,-13.926720,5.830801,11.898513,120003.335938,349.885925,10805003.0
1496,11.327806,-97.041641,-14.619123,4.484382,10.950002,122594.578125,404.638153,10805003.0
1497,11.282037,-96.860786,-14.512480,4.840896,11.609942,119184.578125,368.502167,10805003.0
1498,11.296329,-96.748512,-14.401352,5.321761,11.691708,117609.601562,381.319824,10805003.0
